# LangChain DatetimeOutputParser 예제: 날짜 정보 추출
`ChatOpenAI`와 `DatetimeOutputParser`를 사용해 텍스트에서 날짜/시간을 구조화된 형식으로 추출합니다.

In [ ]:
# 필수 라이브러리 설치
#%pip install python-dateutil

In [10]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:10])

gsk_LbVeBV


In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import DatetimeOutputParser
from datetime import datetime

In [12]:
print(datetime.today())

# 출력 파서 초기화 (시간대 포함 가능)
datetime_parser = DatetimeOutputParser()
format_instructions = datetime_parser.get_format_instructions()

print("날짜 출력 형식 지시사항:")
print(format_instructions)

2025-09-22 09:28:34.267366
날짜 출력 형식 지시사항:
Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 2023-07-04T14:30:00.000000Z, 1999-12-31T23:59:59.999999Z, 2025-01-01T00:00:00.000000Z

Return ONLY this string, no other words!


In [13]:
# 프롬프트 템플릿
# 현재 날짜를 명시적으로 프롬프트에 주입
current_date = datetime.now().strftime("%Y-%m-%d")
template = f"""
현재 날짜: {current_date}
다음 텍스트에서 날짜/시간 정보를 추출하세요. 상대적 표현(예: '다음 주 금요일')은 현재 날짜를 기준으로 계산합니다.
텍스트: {{text}}

{{format_instructions}}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)
print(prompt)

input_variables=['text'] input_types={} partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 2023-07-04T14:30:00.000000Z, 1999-12-31T23:59:59.999999Z, 2025-01-01T00:00:00.000000Z\n\nReturn ONLY this string, no other words!"} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'text'], input_types={}, partial_variables={}, template="\n현재 날짜: 2025-09-22\n다음 텍스트에서 날짜/시간 정보를 추출하세요. 상대적 표현(예: '다음 주 금요일')은 현재 날짜를 기준으로 계산합니다.\n텍스트: {text}\n\n{format_instructions}\n"), additional_kwargs={})]


In [14]:
# 환경변수에서 OpenAI API 키 설정 (실제 사용시 주석 해제)
# import os
# os.environ["OPENAI_API_KEY"] = "your-api-key"

# 모델 초기화 (temperature=0.1로 설정해 정확한 날짜 출력 강조)
#model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.1
)

In [15]:
current_date = datetime.now().strftime("%Y-%m-%d (%A)")
current_date

'2025-09-22 (Monday)'

In [16]:
# 테스트 데이터 (다양한 날짜 형식 포함)
texts = [
    "회의는 2025년 9월 26일 오후 2시에 예정되어 있습니다.",
    f"현재 날짜는 {current_date}입니다. 프로젝트 마감일은 다음주 목요일입니다.",
    "행사 시작: 9/30/2025 18:00 KST",
    "3일 후에 시스템 점검이 진행됩니다.",
    "지난주 금요일 정오에 전시회가 있었습니다"
]
texts

['회의는 2025년 9월 26일 오후 2시에 예정되어 있습니다.',
 '현재 날짜는 2025-09-22 (Monday)입니다. 프로젝트 마감일은 다음주 목요일입니다.',
 '행사 시작: 9/30/2025 18:00 KST',
 '3일 후에 시스템 점검이 진행됩니다.',
 '지난주 금요일 정오에 전시회가 있었습니다']

In [17]:
# 체인 실행 및 결과 출력
chain = prompt | model | datetime_parser

for text in texts:
    print(f"\n원본 텍스트: {text}")
    output = chain.invoke({"text": text})
    print(f"추출된 날짜: {output.strftime('%Y-%m-%d %H:%M:%S %Z')}")


원본 텍스트: 회의는 2025년 9월 26일 오후 2시에 예정되어 있습니다.


BadRequestError: Error code: 400 - {'error': {'message': 'Organization has been restricted. Please reach out to support if you believe this was in error.', 'type': 'invalid_request_error', 'code': 'organization_restricted'}}

## 예상 출력 결과
```
원본 텍스트: 회의는 2023년 11월 15일 오후 2시에 예정되어 있습니다.
추출된 날짜: 2023-11-15 14:00:00 UTC

원본 텍스트: 프로젝트 마감일은 다음 주 금요일입니다.
추출된 날짜: 2023-11-17 00:00:00 UTC  # (현재 날짜가 2023-11-10인 경우)

원본 텍스트: 행사 시작: 12/25/2023 18:00 KST
추출된 날짜: 2023-12-25 09:00:00 UTC  # KST → UTC 변환

원본 텍스트: 3일 후에 시스템 점검이 진행됩니다.
추출된 날짜: 2023-11-13 00:00:00 UTC  # (현재 날짜가 2023-11-10인 경우)
```

### 추가 활용: 이벤트 일정 추출
여러 날짜가 포함된 텍스트에서 이벤트 목록을 추출하는 예제입니다.

In [15]:
# 이벤트 추출용 프롬프트
current_date = datetime.now().strftime("%Y-%m-%d")

event_template = """
현재 날짜: {current_date}
다음 텍스트에서 모든 이벤트의 날짜/시간을 추출하세요. 각 이벤트는 이름과 날짜를 포함해야 합니다.
텍스트: {text}

출력 형식:
- 이벤트명: [이름]
- 날짜: [YYYY-MM-DD HH:MM:SS]
"""

event_prompt = ChatPromptTemplate.from_template(event_template)
event_chain = event_prompt | model

# 예시 텍스트 (여러 이벤트 포함)
event_text = """
12월 10일에 크리스마스 마켓이 열리고, 12월 24일에는 크리스마스 이브 파티가 있습니다.
또한 1월 1일 00:00에 새해 카운트다운이 진행될 예정입니다.
"""

print(event_chain.invoke({"current_date":current_date, "text": event_text}).content)

이 텍스트에서 이벤트 날짜/시간을 다음과 같이 추출했습니다:

- 이벤트명: 크리스마스 마켓
- 날짜: 2025-12-10 00:00:00

- 이벤트명: 크리스마스 이브 파티
- 날짜: 2025-12-24 00:00:00

- 이벤트명: 새해 카운트다운
- 날짜: 2025-01-01 00:00:00


## 예상 출력
```
- 이벤트명: 크리스마스 마켓
- 날짜: 2023-12-10 00:00:00

- 이벤트명: 크리스마스 이브 파티
- 날짜: 2023-12-24 00:00:00

- 이벤트명: 새해 카운트다운
- 날짜: 2024-01-01 00:00:00
```